In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import sklearn### machine learning library
import cv2
from sklearn.metrics import confusion_matrix, roc_curve### metrics
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import (GlobalAveragePooling2D, Activation, MaxPooling2D, Add, Conv2D, MaxPool2D, Dense,
                                     Flatten, InputLayer, BatchNormalization, Input, Embedding, Permute,
                                     Dropout, RandomFlip, RandomRotation, LayerNormalization, MultiHeadAttention,
                                     RandomContrast, Rescaling, Resizing, Reshape)
from tensorflow.keras.losses import BinaryCrossentropy,CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy,TopKCategoricalAccuracy, CategoricalAccuracy, SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers  import L2, L1
from tensorflow.keras.applications import VGG16
import warnings
warnings.filterwarnings('ignore')
print(tf.__version__)

In [ ]:
train_directory = "/kaggle/input/emotion-detection-fer/train"
val_directory = "/kaggle/input/emotion-detection-fer/test"

In [ ]:
classes = ["angry", "disgusted","fearful","happy","neutral", "sad","surprised"]

In [ ]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_directory,
    labels='inferred',
    label_mode='categorical',
    class_names= classes,
    color_mode='rgb',
    batch_size=32,
    image_size=(48, 48),
    shuffle=True,
    seed=99,
)

In [ ]:
val_dataset = tf.keras.utils.image_dataset_from_directory(
    val_directory,
    labels='inferred',
    label_mode='categorical',
    class_names= classes,
    color_mode='rgb',
    batch_size=1,
    image_size=(48,48),
    shuffle=True,
    seed=99,
)

In [ ]:
training_dataset = (
    train_dataset
    .prefetch(tf.data.AUTOTUNE)
)

In [ ]:
validation_dataset = (
    val_dataset
    .prefetch(tf.data.AUTOTUNE)
)

In [ ]:
# Load VGG16 model pre-trained on ImageNet data, without top classification layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))


In [ ]:
# Add custom layers on top of VGG16
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Reduce spatial dimensions
x = Dense(1024, activation='relu')(x)  # Add a fully connected layer
predictions = Dense(7, activation='softmax')(x)  # Final output layer for 7 classes

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
model.summary(_)

In [ ]:
loss_function = CategoricalCrossentropy()

In [ ]:
metrics = [CategoricalAccuracy(name = "accuracy"), TopKCategoricalAccuracy(k=4, name = "top_k_accuracy")]

In [ ]:
model.compile(
  optimizer = Adam(learning_rate = 1e-3),
  loss = loss_function,
  metrics = metrics,
)

In [ ]:
history = model.fit(
  training_dataset,
  validation_data = validation_dataset,
  batch_size= 32,

  epochs = 20,
  verbose = 1
)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train_loss', 'val_loss'])
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train_accuracy', 'val_accuracy'])
plt.show()

In [ ]:
model.save('face_detection_model.h5')